In [1]:
import torch as t
import sys
sys.path.append('../')
from utils import get_score
import glob
import torch as t 
import numpy as np
import json
import time


In [9]:
label_path =   '/home/a/code/pytorch/zhihu/ddd/labels.json'
test_data_path='/home/a/code/pytorch/zhihu/ddd/test.npz' 
index2qid = np.load(test_data_path)['index2qid'].item()
with open(label_path) as f: 
      labels_info = json.load(f)
qid2label = labels_info['d']
label2qid = labels_info['id2label']

In [4]:
data1="/data_ssd/zhihu/result/search_result/test_all_score.pth"
data2="/data_ssd/zhihu/result/search_result/test_all_first.pth"
result1=t.load(data1).float().topk(5,1)[1]
result2=t.load(data2).float().topk(5,1)[1]

result=t.load(data1).float().topk(5,1)[1]

In [5]:
print t.sum(result2[:,0]==result[:,0])


212622


In [27]:
import csv
def write_csv(result,index2qid,label2qid,filename):
    #path="/data_ssd/zhihu/result/search_result/"
    f=open(filename+".csv", "wa")
    csv_writer = csv.writer(f, dialect="excel")
    rows=[0 for _ in range(result.size(0))]
    for i in range(result.size(0)):
        tmp=result[i].sort(dim=0,descending=True)
        tmp=tmp[1][:5]
        row=[index2qid[i]]+[label2qid[str(int(i_))] for i_ in tmp]
        if i%50000==0:
            print row
        rows[i]=row
    csv_writer.writerows(rows)

In [55]:
t.from_numpy

TypeError: 'module' object is not callable

In [7]:
def merge(file1,file2):
    '''
    file1:score
    file2:first
    '''
    Num=file1.size(0)
    res=t.zeros(file1.size())
    for ii in range(Num):
        tmp=file1[ii]
        if file2[ii,0] not in tmp:
            res[ii,0]=file2[ii,0]
            res[ii,1:]=tmp[1:]
        else:
            if file1[ii,0]==file2[ii,0]:
                res[ii]=file1[ii]
            else:
                res[ii,0]=file2[ii,0]
                aaa=np.array([r for r in tmp if r!=file1[ii,0]])
                res[ii,1:]=t.from_numpy(aaa)
    return res
                
                

In [33]:
result2[0]


 1422
  808
  714
 1232
  609
[torch.LongTensor of size 5]

In [26]:
rer[0]


 1422
  808
 1232
  714
  609
[torch.FloatTensor of size 5]

In [21]:
print result.size()

torch.Size([217360, 5])


In [31]:
rows = range(rer.size(0))
for ii,item in enumerate(rer):
    rows[ii] = [index2qid[ii]] + [label2qid[str(int(_))] for _ in item ]
    if ii%50000==0:
        print rows[ii]
import csv
with open('0.43082.csv','w') as f:
    writer = csv.writer(f)
    writer.writerows(rows)

[u'6215603645409872328', u'4610596224687453206', u'-7506384235581390893', u'2919247920214845195', u'-6839713564940654454', u'-5932391056759866388']
[u'2558136278626495418', u'3040328401474530651', u'5947293336854386967', u'2587540952280802350', u'-2299947672181109599', u'6006627476560013656']
[u'-5497959296411633559', u'-7057974542385626232', u'7263346613600732510', u'7592036674792056069', u'-8261875436896588399', u'-2244015987817045493']
[u'8330621920012103679', u'2515257722796787281', u'6179724419779689236', u'1790630374868647951', u'7580468821069307570', u'-2523521411748733187']
[u'909270500917796775', u'-1945278234078776687', u'521166281704794588', u'5953579394093512194', u'-6748914495015758455', u'7476760589625268543']


In [17]:
result[:,0]=result2[:,0]
print t.sum(result1[:,0]==result[:,0])

212622


In [25]:
a=rer[:,0]
b=result[:,0]
print a.size()
b.size()

torch.Size([217360])


torch.Size([217360])

In [6]:
true_labels = [qid2label[index2qid[2999967-200000+ii]] for ii in range(200000)]

In [81]:
predict_label_and_marked_label_list = [[_1,_2] for _1,_2 in zip(rer,true_labels)]
score,_,_,_ = get_score(predict_label_and_marked_label_list)
score

0.43082848236815924

In [4]:
files = glob.glob('../checkpoints/result/*val.pth')
r2=0
for file in files:
    if 'MultiModel' not in file:
        print file
        r2+=t.load(file)

../checkpoints/result/LSTMText0.4119_word_val.pth
../checkpoints/result/RCNN_0.4037_char_val.pth
../checkpoints/result/LSTMText0.4031_char_val.pth
../checkpoints/result/RCNN_0.4115_word_val.pth
../checkpoints/result/DeepText0.4103_word_val.pth
../checkpoints/result/FastText3_word_0.409160_val.pth
../checkpoints/result/CNNText_tmp0.4109_word_val.pth
../checkpoints/result/CNNText_tmp0.4024_char_val.pth


In [7]:
true_labels = [qid2label[index2qid[2999967-200000+ii]] for ii in range(len(r2))]

In [9]:
result=r2.topk(5,1)[1]
predict_label_and_marked_label_list = [[_1,_2] for _1,_2 in zip(result,true_labels)]
score,_,_,_ = get_score(predict_label_and_marked_label_list)
score

0.42763755264432574

In [10]:
del r2

In [11]:
files = glob.glob('../checkpoints/result/tmp/*val.pth')
r_aug=0
for file in files:
    if 'MultiModel' not in file and 'weight5' not in file:
        print file
        r_aug+=t.load(file)

../checkpoints/result/tmp/DeepText_word_0.410051_aug_val.pth
../checkpoints/result/tmp/RCNN0.41344_aug_word_val.pth
../checkpoints/result/tmp/FastText3_word_weight1_0.40853_val.pth
../checkpoints/result/tmp/inception0.41254_aug_word_val.pth
../checkpoints/result/tmp/MultiCNNTextBNDeep_aug_char_0.38738_val.pth
../checkpoints/result/tmp/RCNN0.39854_aug_char_val.pth
../checkpoints/result/tmp/LSTMText0.41368_aug_word_val.pth


In [12]:
result=r_aug.topk(5,1)[1]
predict_label_and_marked_label_list = [[_1,_2] for _1,_2 in zip(result,true_labels)]
score,_,_,_ = get_score(predict_label_and_marked_label_list)
score

0.42659117805840374

In [13]:
del r_aug

In [14]:
files = glob.glob('../checkpoints/result/tmp/*val.pth')
r_multi=0
for file in files:
    if 'MultiModel' in file :
        print file
        r_multi+=t.load(file)

../checkpoints/result/tmp/MultiModelallfast_0.419088_aug_val.pth
../checkpoints/result/tmp/MultiModel_0.4171859_val.pth
../checkpoints/result/tmp/MultiModel2_0.42560_val.pth
../checkpoints/result/tmp/MultiModel_0.42169_val.pth
../checkpoints/result/tmp/MultiModelall_0.4235_aug_val.pth
../checkpoints/result/tmp/MultiModel2w2c_0.4213_val.pth
../checkpoints/result/tmp/MultiModelallfast_0.41652_aug_val.pth
../checkpoints/result/tmp/MultiModel_0.41987_word_val.pth


In [16]:
result=r_multi.topk(5,1)[1]
predict_label_and_marked_label_list = [[_1,_2] for _1,_2 in zip(result,true_labels)]
score,_,_,_ = get_score(predict_label_and_marked_label_list)
del r_multi
score

0.4302698214561717

In [17]:
files = glob.glob('../checkpoints/result/tmp/*val.pth')
r_5=0
for file in files:
    if 'weight5' in file :
        print file
        r_5+=t.load(file)

../checkpoints/result/tmp/MultiCNNTextBNDeep_word_weight5_0.409409_val.pth
../checkpoints/result/tmp/LSTMText_0.41230_weight5_val.pth
../checkpoints/result/tmp/RCNN_0.41226_weight5_val.pth
../checkpoints/result/tmp/inception_0.409912_weight5_val.pth
../checkpoints/result/tmp/FastText3_word_weight5_0.407992_val.pth


In [19]:
result=r_5.topk(5,1)[1]
predict_label_and_marked_label_list = [[_1,_2] for _1,_2 in zip(result,true_labels)]
score,_,_,_ = get_score(predict_label_and_marked_label_list)
del r_5
score

0.4187042397507684

In [22]:
files = glob.glob('/data_ssd/zhihu/result/search_result/*_score.pth')
files

['/data_ssd/zhihu/result/search_result/search_multimodel_result_score.pth',
 '/data_ssd/zhihu/result/search_result/search_nomulti_weight1_result_score.pth',
 '/data_ssd/zhihu/result/search_result/search_noaug_result_score.pth',
 '/data_ssd/zhihu/result/search_result/search_weight5_result_score.pth']

In [83]:
r_re=18*t.load(files[0])+1.495*t.load(files[1])+t.load(files[2])+0.15*t.load(files[3])

In [84]:
result=r_re.topk(5,1)[1]
predict_label_and_marked_label_list = [[_1,_2] for _1,_2 in zip(result,true_labels)]
score,_,_,_ = get_score(predict_label_and_marked_label_list)
del r_re
score

0.43059689877126467

18 1.0 1 0.15    0.4305411700375018
18 1.2 1 0.15    0.4305619687580149
18 1.4 1 0.15    0.4305940768722144
18 1.45 1 0.15   0.43059294166321205
18 1.5 1 0.15    0.4306030584472225
18 1.55 1 0.15   0.43060288677521397
18 1.6 1 0.15    0.43059537396039643
18 1.8 1 0.15    0.4305601537228334
18 2.5 1 0.15    0.4304910145968311

10.0 1.5 1 0.15    0.43052812976083576
15.0 1.5 1 0.15    0.43058234950124885
17.0 1.5 1 0.15    0.4306006464205759
18.0 1.5 1 0.15    0.4306030584472225
18.4 1.5 1 0.15    0.43059276943956926
18.5 1.5 1 0.15    0.43059091975216024
19.0 1.5 1 0.15    0.4305840952366161
20.0 1.5 1 0.15    0.4305720334469683
30.0 1.5 1 0.15    0.4305376536701278

In [1]:
result=(r2+r_aug*1.1+r_multi*14+r_5*0.15).topk(5,1)[1]

NameError: name 'r2' is not defined

In [61]:
predict_label_and_marked_label_list = [[_1,_2] for _1,_2 in zip(result,true_labels)]
score,_,_,_ = get_score(predict_label_and_marked_label_list)
score

0.4307485778331298

In [8]:
    rows = range(result.size(0))
    for ii,item in enumerate(result):
        rows[ii] = [index2qid[ii]] + [label2qid[str(_)] for _ in item ]
    import csv
    with open('result0.43074.csv','w') as f:
        writer = csv.writer(f)
        writer.writerows(rows)

In [10]:

import csv
line_no=217360
rate = 0.003
with open('result0.43074.csv') as f:
    lines = [line.strip().split(',') for line in f.readlines()]
    num = rate*line_no
    stride = int(1/rate)
    for line in lines[::stride]:
        line[1:]=[-1,-1,-1,-1,-1]
with open('result0.43074*0.997.csv','w') as f:
    writer = csv.writer(f)
    writer.writerows(lines)  
        

In [3]:
lines[0]

['6215603645409872328',
 '4610596224687453206',
 '-7506384235581390893',
 '-6839713564940654454',
 '2919247920214845195',
 '-5932391056759866388\r\n']

In [7]:
0.429477037467622/0.99

0.43381518936123437

In [19]:
predict_label_and_marked_label_list = [[_1,_2] for _1,_2 in zip(result,true_labels)]
score,_,_,_ = get_score(predict_label_and_marked_label_list)
score

0.4285008976173052

In [ ]:
def target(args):
        w1,w2,w3 = args
        r = a + b*w1 +c*w2 + d*w3
        result = r.topk(5,1)[1]
        predict_label_and_marked_label_list = [[_1,_2] for _1,_2 in zip(result,true_labels)]
        score,_,_,rrrrrrr = get_score(predict_label_and_marked_label_list)
        print (args,score,_)#list_space = [hp.uniform('a',0,1),hp.uniform('b',0,1)]
        return -rrrrrrr[0]

In [16]:
d=results[0]

In [20]:
d[-1].sum()

2.9034742719438373

In [57]:
predict_label_and_marked_label_list = [[_1,_2] for _1,_2 in zip(result,true_labels)]
score,_,_,_ = get_score(predict_label_and_marked_label_list)
score

0.4267781957592053

0.42655971790607233

In [19]:
score

0.4253965699709525

In [ ]:
#coding:utf8


label_path =   '/mnt/7/zhihu/ieee_zhihu_cup/data/labels.json'
test_data_path='/mnt/7/zhihu/ieee_zhihu_cup/data/test.npz' 
def ensamble(file1,file2,label_path=label_path,     test_data_path=test_data_path,result_csv=None):

    if result_csv is None:
        
        result_csv = time.strftime('%y%m%d_%H%M%S.csv')
    a = t.load(file1)
    b = t.load(file2)
    r = 9.0*a+b
    result = r.topk(5,1)[1]
    
    index2qid = np.load(test_data_path)['index2qid'].item()
    with open(label_path) as f:   label2qid = json.load(f)['id2label']
    rows = range(result.size(0))
    for ii,item in enumerate(result):
        rows[ii] = [index2qid[ii]] + [label2qid[str(_)] for _ in item ]
    import csv
    with open(result_csv,'w') as f:
        writer = csv.writer(f)
        writer.writerows(rows)
    

if __name__ == '__main__':
    import fire
    fire.Fire()

files = ['CNNText_tmp0.4024_char_test.pth',
 'CNNText_tmp0.4024_char_val.pth',
 'DeepText0.4103_word_test.pth',
 'Inception0.4110_word.pth',
 'LSTMText0.4119_word.pth',
 'LSTMText0.4031_char_test.pth',
 'LSTMText0.4119_word_test.pth',
 'DeepText0.4103_word_val.pth',
 'CNNText_tmp0.4109_word_val.pth',
 'LSTMText0.4119_word_val.pth',
 'RCNN_0.4115_word_test.pth',
 'RCNN_0.4037_char_val.pth',
 'LSTMText0.4031_char_val.pth',
 'RCNN_0.4115_word_val.pth',
 'RCNN_0.4037_char_test.pth',
 'CNNText_tmp0.4109_word_test.pth']
